In [3]:
from z3 import *
import math

## Utils

### At most/least one & exactly one

In [4]:
def at_least_one_seq(bool_vars):
    return Or(bool_vars)

def at_most_one_seq(x, name):
    # bool_vars renamed to x for simplicity
    n = len(x)
    s = [Bool(f"s_{i}_{name}") for i in range(n-1)]     # "name" is used in order to create a
                                                        # unique set of variables s for each constraint
                                                        # of the kind at_most_one_seq, so that they do not overlap 
                                                        # for different constraints!
                                                        # N.B. len(s) = n-1 != n = len (x)
    clauses = []
    clauses.append(Or(Not(x[0]), s[0]))                 # x[0] -> s[0] (s[i] modeled as: s[i] is true if the SUM UP TO index i IS 1!) i.e. x_1 -> s_1 in math notation
    for i in range(1, n-1):
        clauses.append(Or(Not(x[i]), s[i]))             # these two clauses model (x[i] v s[i-1]) -> s[i]
        clauses.append(Or(Not(s[i-1]), s[i]))
        clauses.append(Or(Not(s[i-1]), Not(x[i])))      # this one models s[i-1] -> not x[i]
    clauses.append(Or(Not(s[-1]), Not(x[-1])))          # s[n-2] -> not x[n-1]  i.e. s_(n-1) -> s_n in the mathematical notation (1-based)
    return And(clauses)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

In [5]:
def find_max_digits(s, l): # TODO Idea: optimize number of digits depending on cases
    return math.ceil(math.log2(max(sum(s), max(l))))

In [25]:
def int_to_bin(x, digits=5):  # TODO read: #https://ericpony.github.io/z3py-tutorial/guide-examples.htm
    x_bin = [(x%(2**(i+1)) // 2**i)==1 for i in range(digits-1,-1,-1)]
    return x_bin

In [43]:
def sum(a_bin, b_bin, digits, d_bin):
    #slide SAT-I
    c = [Bool(f"c_{k}") for k in range(digits+1)]
    c[0] = False
    c[-1] = False

    clauses = []
    for k in range(digits-1,-1,-1):
        # clauses.append((a_bin[k] == b_bin[k]) == (c[k+1] == d_bin[k]))    #TODO: check if it is faster
        clauses.append(d_bin[k] == Or(And(a_bin[k], b_bin[k], c[k+1]), And(a_bin[k], Not(b_bin[k]), Not(c[k+1])), 
                                   And(Not(a_bin[k]), b_bin[k], Not(c[k+1])), And(Not(a_bin[k]), Not(b_bin[k]), c[k+1])))
        clauses.append(c[k] == Or(And(a_bin[k],b_bin[k]), And(a_bin[k],c[k+1]), And(b_bin[k],c[k+1])))

    return And(clauses)

s = Solver()
s.add(sum(int_to_bin(11), int_to_bin(8), 5, int_to_bin(19)))
print(s.check())

sat


In [44]:
for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 8):
            s = Solver()
            s.add(sum(int_to_bin(i), int_to_bin(j), 5, int_to_bin(k)))
            print(i, '+', j, '=', k, '-->', s.check())

1 + 1 = 1 --> unsat
1 + 1 = 2 --> sat
1 + 1 = 3 --> unsat
1 + 1 = 4 --> unsat
1 + 1 = 5 --> unsat
1 + 1 = 6 --> unsat
1 + 1 = 7 --> unsat
1 + 2 = 1 --> unsat
1 + 2 = 2 --> unsat
1 + 2 = 3 --> sat
1 + 2 = 4 --> unsat
1 + 2 = 5 --> unsat
1 + 2 = 6 --> unsat
1 + 2 = 7 --> unsat
1 + 3 = 1 --> unsat
1 + 3 = 2 --> unsat
1 + 3 = 3 --> unsat
1 + 3 = 4 --> sat
1 + 3 = 5 --> unsat
1 + 3 = 6 --> unsat
1 + 3 = 7 --> unsat
2 + 1 = 1 --> unsat
2 + 1 = 2 --> unsat
2 + 1 = 3 --> sat
2 + 1 = 4 --> unsat
2 + 1 = 5 --> unsat
2 + 1 = 6 --> unsat
2 + 1 = 7 --> unsat
2 + 2 = 1 --> unsat
2 + 2 = 2 --> unsat
2 + 2 = 3 --> unsat
2 + 2 = 4 --> sat
2 + 2 = 5 --> unsat
2 + 2 = 6 --> unsat
2 + 2 = 7 --> unsat
2 + 3 = 1 --> unsat
2 + 3 = 2 --> unsat
2 + 3 = 3 --> unsat
2 + 3 = 4 --> unsat
2 + 3 = 5 --> sat
2 + 3 = 6 --> unsat
2 + 3 = 7 --> unsat
3 + 1 = 1 --> unsat
3 + 1 = 2 --> unsat
3 + 1 = 3 --> unsat
3 + 1 = 4 --> sat
3 + 1 = 5 --> unsat
3 + 1 = 6 --> unsat
3 + 1 = 7 --> unsat
3 + 2 = 1 --> unsat
3 + 2 = 2 --> 

In [ ]:
def LinearInteger(c_i, s, l_i, i, n, digits):
    """ c_i: a[i,:] in model 1
        s: list of sizes of objects
        l_i: load of courier i
        i: index of courier (doing the job of name)
        digits: number of digits required to represent l_i and numbers in s
    """
    # matrix containing temporary results of sum
    d = [[Bool(f"d_{j}_{k}_{i}") for k in range(digits)] for j in range(n)]

    clauses = []
    # TODO: aggiungere "if c_i[j] = 1, fai la roba sotto, altrienti d_j == d_j-1"
    for j in range(n):
        clauses.append(sum(d[j-1, :], s[j], d[j, :])) # temporary result d_j-1 + s_j = (imposed) d_j

    # TODO: aggiungere a clause che d_n <= l_i
    return And(clauses)



## Model 1

In [5]:
def multiple_couriers_planning_1(m, n, l, s, D):
    ## VARIABLES

    # a for assignments
    a = [[Bool(f"a_{i}_{j}") for j in range(n)] for i in range(m)]
    # a_ij = 1 indicates that courier i takes object j
    # O(m * n) vars

    # r for routes
    r = [[[Bool(f"r_{k}_{j}_{i}") for k in range(n)] for j in range(n)] for i in range(m)]  
    # r_kji = 1 indicates that courier i delivers object j as its k-th delivery
    # O(m * n^2) vars

    solver = Solver()


    ## CONSTRAINTS
    # Constraint 1: every object is assigned to one and only one courier
    for j in range(n):
        solver.add(exactly_one_seq([a[i][j] for i in range(m)], f"assignment_{j}"))
    
    # Interesting question: see discussion forum (distance func. property)
    # -> answer: the distance is quasimetric => we can add implicit constraint "every courier has at least 1 package to deliver"
    # TODO: check if it actually speeds up execution
    

